In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [ ]:
MITDB_RESULT = "results_evaluation_localization/mitdb_resnet18_7_bs32_lr5e-2_wd1e-4_ep20/_final_results.csv"
mitdb_df_l = pd.read_csv(MITDB_RESULT, index_col=0).drop(["random_baseline_results", "random_baseline_results_absolute"])
SVDB_RESULT = "results_evaluation_localization/svdb_resnet18_7_bs32_lr1e-2_wd1e-4_ep20/_final_results.csv"
svdb_df_l = pd.read_csv(SVDB_RESULT, index_col=0).drop(["random_baseline_results", "random_baseline_results_absolute"])
INCARTDB_RESULT = "results_evaluation_localization/incartdb_resnet18_7_bs32_lr1e-3_wd1e-4_ep20/_final_results.csv"
incartdb_df_l = pd.read_csv(INCARTDB_RESULT, index_col=0).drop(["random_baseline_results", "random_baseline_results_absolute"])
ICENTIA_RESULT = "results_evaluation_localization/icentia11k_resnet18_7_bs32_lr1e-3_wd1e-4_ep20/_final_results.csv"
icentia_df_l = pd.read_csv(ICENTIA_RESULT, index_col=0).drop(["random_baseline_results", "random_baseline_results_absolute"])

### 6.1. Ranking Consistency

In [ ]:
RESULT_DIR = "./figures_correlation_localization"
os.makedirs(RESULT_DIR, exist_ok=True)

# method = "pearson"
# method = "spearman"
method = "kendall"

In [ ]:
plt.rcParams['font.family'] = 'Arial'
plt.rcParams.update({'font.size': 16})

In [ ]:
title_dict = {
    "attribution_localization": "Attribution Localization",
    "relevance_rank_accuracy": "Relevance Rank Accuracy",
    "pointing_game": "Pointing Game",
    "auc": "AUC",
}

In [ ]:
metrics_l = ["attribution_localization", "relevance_rank_accuracy", "pointing_game", "auc"]

dfs_l = {
    "mitdb": mitdb_df_l,
    "svdb": svdb_df_l,
    "incartdb": incartdb_df_l,
    "icentia": icentia_df_l,
}

metric_results_l = {metric: pd.concat([mitdb_df_l[f"{metric}_mean"].rename("MITDB"), svdb_df_l[f"{metric}_mean"].rename("SVDB"), incartdb_df_l[f"{metric}_mean"].rename("INCARTDB"), icentia_df_l[f"{metric}_mean"].rename("ICENTIA11K")], axis=1) for metric in metrics_l}

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 17.5))
cmap = matplotlib.colormaps.get_cmap("viridis")

for idx, metric in enumerate(metrics_l):
    corr_matrix = metric_results_l[metric].corr(method=method)
    # corr_matrix = metric_results_l[metric][[not ("absolute" in s) for s in metric_results_l[metric].index]].corr(method=method)
    up_triang = np.triu(np.ones_like(corr_matrix)).astype(bool)
    ax = axs[idx//2][idx%2]
    sns.heatmap(corr_matrix, vmin=0, vmax=1, cmap="viridis", fmt=".3f", annot=True, mask=up_triang, ax=ax, cbar=False, annot_kws={"size": 22})
    mean_corr = np.mean(corr_matrix.to_numpy()[up_triang==False])
    ax.text(1.75, 1.5, f"{mean_corr:.3f}", fontsize=26, color=("w" if mean_corr <= 0.712 else "k"), bbox=dict(facecolor=cmap(mean_corr), boxstyle="round,pad=0.5", edgecolor="none"))
    ax.set_title(title_dict[metric], fontsize=30)

fig.tight_layout()
fig.subplots_adjust(hspace=0.22)

plt.savefig(os.path.join(RESULT_DIR, f"corrleation_{method}.png"))

In [ ]:
fig = plt.figure(figsize=(5,15))
sns.heatmap([[0,1],[1,0]], vmin=0, vmax=1, cmap="viridis", fmt=".3f", annot=True, cbar=True)
plt.tight_layout()
plt.savefig(os.path.join(RESULT_DIR, f"cbar.png"))

### 6.2. Inter-metric correlation

In [ ]:
RESULT_DIR = "figures_metrics_correlation_localization"
os.makedirs(RESULT_DIR, exist_ok=True)

In [ ]:
# method = "pearson"
method = "spearman"
# method = "kendall"

In [ ]:
result_dict = {}
datasets = ["MITDB", "SVDB", "INCARTDB", "ICENTIA11K"]
for dataset in datasets:        
    dataset_result_l = pd.concat([metric_results_l[metric][dataset] for metric in metrics_l], axis=1, keys=metrics_l)
    result_dict[dataset] =dataset_result_l.rename(columns={"attribution_localization": "AL", "relevance_rank_accuracy": "RRA", "pointing_game": "PG", "auc": "AUC"})

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 17.5))
cmap = matplotlib.colormaps.get_cmap("viridis")

for idx, dataset in enumerate(datasets):
    corr_matrix = result_dict[dataset].corr(method=method)
    # corr_matrix = result_dict[dataset][[not ("absolute" in s) for s in result_dict[dataset].index]].corr(method=method)

    up_triang = np.triu(np.ones_like(corr_matrix)).astype(bool)
    ax = axs[idx%2][idx//2]
    sns.heatmap(corr_matrix, vmin=0, vmax=1, cmap="viridis", fmt=".3f", annot=True, mask=up_triang, ax=ax, cbar=False, annot_kws={"size": 22})
    ax.set_title(dataset, fontsize=30)

fig.tight_layout()
fig.subplots_adjust(hspace=0.22)
plt.savefig(os.path.join(RESULT_DIR, f"metric_corrleation_{method}.png"))


In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 8.5))
cmap = matplotlib.colormaps.get_cmap("viridis")

total_corr_matrix = np.zeros((4,4), dtype=np.float64)

for idx, dataset in enumerate(datasets):
    corr_matrix = result_dict[dataset].corr(method=method)
    total_corr_matrix += corr_matrix
    # corr_matrix = result_dict[dataset][[not ("absolute" in s) for s in result_dict[dataset].index]].corr(method=method)

total_corr_matrix /= 4

up_triang = np.triu(np.ones_like(total_corr_matrix)).astype(bool)
sns.heatmap(total_corr_matrix, vmin=0, vmax=1, cmap="viridis", fmt=".3f", annot=True, mask=up_triang, cbar=False, annot_kws={"size": 22})
axs.set_title("Average", fontsize=30)

fig.tight_layout()
fig.subplots_adjust(hspace=0.22)
plt.savefig(os.path.join(RESULT_DIR, f"average_metric_corrleation_{method}.png"))